# Support Vector Machine - Hourly Taxi Demand Prediction

<hr style="border: 1px solid #8888;">

In [6]:
# import all necessary packages
import pandas as pd
import numpy as np
import json
import datetime
import warnings
from pandas.core.common import SettingWithCopyWarning

#warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
#import seaborn as sns
#import matplotlib as mlp
import matplotlib.pyplot as plt
#from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.svm import SVR
import math
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from scipy.stats import loguniform

In [7]:
# import datasets
df_h_comar = pd.read_csv("data/datasets/df_h_comar.csv")
df_h_hexlow = pd.read_csv("data/datasets/df_h_hexlow.csv")
df_h_hexmed = pd.read_csv("data/datasets/df_h_hexmed.csv")
df_h_hexhig = pd.read_csv("data/datasets/df_h_hexhig.csv")

In [8]:
df_h_comar.drop(columns = ['date_start'],inplace = True)
df_h_hexlow.drop(columns = ['date_start'],inplace = True)
df_h_hexmed.drop(columns = ['date_start'],inplace = True)
df_h_hexhig.drop(columns = ['date_start'],inplace = True)

In [9]:
# defining categorical and numeric features of dfPhiladelphia

categoric = ['start_stamp', 'Pickup_Community_Area','dayOfWeek','start_time_month','start_time_day','start_time_week',
             'isHoliday','description','isRushhour', 'season']
numeric = ['temperature_celsius','wind_speed','wind_direction','humidity','pressure']

In [10]:
# function for normalize numeric and encode categorical features and for create pipeline

def pipeline_for_prediction(categoric, numeric, model):
    
    numeric_transformer = Pipeline(steps=[("standard_scaler", StandardScaler())])
    categorical_transformer = Pipeline(
        steps=[("one_hot_encoder", OneHotEncoder(handle_unknown="ignore"))]
    )
    preprocessor = ColumnTransformer(
        transformers=[
            ("numerical scaler", numeric_transformer, numeric),
            ("one hot encoder", categorical_transformer, categoric),
        ]
    )
    pipeline = Pipeline(
        steps=[("preprocessor", preprocessor), ("model", model)]
    )
    return pipeline

In [11]:
# function for getting different scores for a model

def get_prediction_scores(y_true, y_predicted):
    print("MODEL SCORES:")
    print(f"MAE: {metrics.mean_absolute_error(y_true, y_predicted): .3f}")
    print(f"MSE: {metrics.mean_squared_error(y_true, y_predicted): .3f}")
    print(f"RMSE: {math.sqrt(metrics.mean_squared_error(y_true, y_predicted)): .3f}")
    print(f"Accuracy:", round((1-(metrics.mean_absolute_error(y_true, y_predicted)/df_h_comar["numOfTaxis_area"].mean()))*100,2), "%")
    print(f"R2: {100 * metrics.r2_score(y_true, y_predicted): .3f} %")
    print(f"Max Residual Error: {metrics.max_error(y_true, y_predicted): .3f}")

In [21]:
# function for creating pipeline and fitting model (created by the pipeline), predict and printing scores

def pipeline_fit_predict(reg, categoric, numeric, x_train, y_train, x_val, y_val):
    pipeline = pipeline_for_prediction(categoric, numeric, reg)
    pipeline.fit(x_train, y_train)
    y_predict = pipeline.predict(x_val)
    get_prediction_scores(y_val, y_predict)

In [ ]:
# function for finding the best hyperparameter by using RandomizedSearchCV and RepeatedStratifiedKFold
"""parameter:
   - pipeline: used pipeline for grid search (the pipeline contains the model)
   - x_val: data set (features) used for grid search
   - y_val: data set (target value) used for grid search
   - model_par: parameters for which the grid search is done
   - score: used score measure 
   - n_iter: how often grid search will be done
   - n_repeats: how often the data set is randomly splitted (by using the same random hyperparameter) in n_splits
   - n_splits: number of splits in RepeatedStratifiedKFold
   - verbose: getting information during the grid search
"""


from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold

def find_best_hyperparameters(pipeline, x_val, y_val, model_par, score, n_iter = 50,  
                                   n_repeats=1, n_splits=5, n_jobs=1, verbose=True): #  n_repeats=3
    
    print(f"Running grid search for the model based on {score}")
    grid_pipeline = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=model_par,
        n_jobs=n_jobs,
        n_iter=n_iter,
        cv=RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42),
        scoring=score,
        random_state=42,
        verbose=verbose,
    )
    grid_pipeline.fit(x_val, y_val)
    print(f"Best {score} Score was: {grid_pipeline.best_score_}")
    print("The best hyper parameters for the model are:")
    print(grid_pipeline.best_params_)

In [ ]:
from sklearn.model_selection import train_test_split

def train_val_test(df,testsize=0.3,valsize=0.2):
    
    #split the data set in (1-testsize)% training set and testsize% testing set
    x_train, x_test, y_train, y_test = train_test_split(df.drop('numOfTaxis_area', axis=1)
                                                    , df['numOfTaxis_area'], 
                                                    test_size=testsize,random_state=42)

    # save the combination of training and validation set in extra variables
    x_train_val = x_train
    y_train_val = y_train

    #split the training data set in 70% training set and 20% validation set to achieve a 50-20-30 split
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=(valsize/(1-testsize)), 
                                                      random_state = 42)
    
    return x_train_val, y_train_val, x_train, y_train, x_val, y_val

In [ ]:
 #split the data set in 70% training set and 30% testing set
    #x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.3,random_state=42)
  #  x_train, x_test, y_train, y_test = train_test_split(df_h_comar.drop('numOfTaxis_area', axis=1)
                                  #                  , df_h_comar['numOfTaxis_area'], 
                                   #                 test_size=0.3,random_state=42)

    # save the combination of training and validation set in extra variables
   # x_train_val = x_train
    #y_train_val = y_train

    #split the training data set in 70% training set and 20% validation set to achieve a 50-20-30 split
    #x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=(0.2/0.7), random_state = 42)

<hr style="border: 1px solid #8888;">

## Hourly Demand - Community Area

### Building the model: Checking for the best kernel

### SVR Kernel: linear

In [24]:
svr_lin = SVR(kernel = 'linear',verbose = 10,cache_size=15000, max_iter=50000)

In [12]:
pipeline = pipeline_fit_predict(svr_lin, categoric, numeric, x_train_val, y_train_val, x_test, y_test)

[LibSVM].................................................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 50000
obj = -3363473.268517, rho = -11.201561
nSV = 95981, nBSV = 95028


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


MODEL SCORES:
MAE:  23.337
MSE:  4498.339
RMSE:  67.070
Accuracy: 40.8 %
R2:  56.381 %
Max Residual Error:  1446.159


### SVR Kernel: polynomial degree 2

In [18]:
svr_poly2 = SVR(kernel = 'poly',degree = 2, verbose = 10,cache_size=15000, max_iter=50000)

In [19]:
pipeline = pipeline_fit_predict(svr_poly, categoric, numeric, x_train_val, y_train_val, x_test, y_test)

[LibSVM].................................................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 50000
obj = -4735355.356241, rho = -6.414818
nSV = 99966, nBSV = 99966


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


MODEL SCORES:
MAE:  26.649
MSE:  7286.713
RMSE:  85.362
Accuracy: 32.39 %
R2:  29.343 %
Max Residual Error:  1635.635


### SVR Kernel: polynomial degree 3

In [13]:
svr_poly3 = SVR(kernel = 'poly',verbose = 10,cache_size=15000, max_iter=50000)

In [14]:
pipeline = pipeline_fit_predict(svr_poly3, categoric, numeric, x_train_val, y_train_val, x_test, y_test)

[LibSVM].................................................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 50000
obj = -5031586.430808, rho = 4.514757
nSV = 99968, nBSV = 99968


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


MODEL SCORES:
MAE:  35.897
MSE:  8870.264
RMSE:  94.182
Accuracy: 8.93 %
R2:  13.987 %
Max Residual Error:  1689.392


### SVR Kernel: rbf

In [16]:
svr_rbf = SVR(kernel = 'rbf',verbose = 10,cache_size=15000, max_iter=50000)

In [17]:
pipeline = pipeline_fit_predict(svr_rbf, categoric, numeric, x_train_val, y_train_val, x_test, y_test)

[LibSVM].................................................WARN: libsvm Solver reached max_iter
optimization finished, #iter = 50000
obj = -4792854.643942, rho = -6.669636
nSV = 99984, nBSV = 99984


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


MODEL SCORES:
MAE:  26.690
MSE:  7261.736
RMSE:  85.216
Accuracy: 32.29 %
R2:  29.585 %
Max Residual Error:  1634.289


<hr style="border: 0.5px solid #8888;">

### Hyperparameter Tuning for the best model

In [15]:
# creating the ranges for model parameter to use in find_best_hyperparameters

model_para = {'model__C':loguniform(1e-1, 1e2),        
                'model__epsilon':loguniform(1e-1, 1e2)}               

In [16]:
svr_lin = SVR(kernel = 'linear',cache_size=15000, max_iter=50000)

In [17]:
pipeline = pipeline_for_prediction(categoric, numeric, svr_lin)

In [18]:
find_best_hyperparameters(pipeline, x_val, y_val, model_para, score = 'neg_mean_absolute_error', verbose=10)

Running grid search for the model based on neg_mean_absolute_error
Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5; 1/50] START model__C=1.3292918943162166, model__epsilon=71.14476009343416


/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[CV 1/5; 1/50] END model__C=1.3292918943162166, model__epsilon=71.14476009343416;, score=-50.789 total time=  11.2s
[CV 2/5; 1/50] START model__C=1.3292918943162166, model__epsilon=71.14476009343416
[CV 2/5; 1/50] END model__C=1.3292918943162166, model__epsilon=71.14476009343416;, score=-50.442 total time=  11.0s
[CV 3/5; 1/50] START model__C=1.3292918943162166, model__epsilon=71.14476009343416
[CV 3/5; 1/50] END model__C=1.3292918943162166, model__epsilon=71.14476009343416;, score=-50.972 total time=  11.0s
[CV 4/5; 1/50] START model__C=1.3292918943162166, model__epsilon=71.14476009343416
[CV 4/5; 1/50] END model__C=1.3292918943162166, model__epsilon=71.14476009343416;, score=-50.549 total time=  11.1s
[CV 5/5; 1/50] START model__C=1.3292918943162166, model__epsilon=71.14476009343416
[CV 5/5; 1/50] END model__C=1.3292918943162166, model__epsilon=71.14476009343416;, score=-51.020 total time=  11.2s
[CV 1/5; 2/50] START model__C=15.702970884055382, model__epsilon=6.2513735745217485


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 1/5; 2/50] END model__C=15.702970884055382, model__epsilon=6.2513735745217485;, score=-23.825 total time= 1.1min
[CV 2/5; 2/50] START model__C=15.702970884055382, model__epsilon=6.2513735745217485


...

/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 4/5; 49/50] END model__C=3.699972431463808, model__epsilon=1.9170041589170652;, score=-23.046 total time= 1.4min
[CV 5/5; 49/50] START model__C=3.699972431463808, model__epsilon=1.9170041589170652


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 5/5; 49/50] END model__C=3.699972431463808, model__epsilon=1.9170041589170652;, score=-23.574 total time= 1.4min
[CV 1/5; 50/50] START model__C=0.11919481947918735, model__epsilon=0.21070472806578233
[CV 1/5; 50/50] END model__C=0.11919481947918735, model__epsilon=0.21070472806578233;, score=-26.635 total time= 1.6min
[CV 2/5; 50/50] START model__C=0.11919481947918735, model__epsilon=0.21070472806578233
[CV 2/5; 50/50] END model__C=0.11919481947918735, model__epsilon=0.21070472806578233;, score=-27.088 total time= 1.6min
[CV 3/5; 50/50] START model__C=0.11919481947918735, model__epsilon=0.21070472806578233
[CV 3/5; 50/50] END model__C=0.11919481947918735, model__epsilon=0.21070472806578233;, score=-26.864 total time= 3.3min
[CV 4/5; 50/50] START model__C=0.11919481947918735, model__epsilon=0.21070472806578233
[CV 4/5; 50/50] END model__C=0.11919481947918735, model__epsilon=0.21070472806578233;, score=-26.582 total time= 1.6min
[CV 5/5; 50/50] START model__C=0.11919481947918735, mod

/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [19]:
svr_lin_daily = SVR(kernel = 'linear',cache_size=15000, max_iter=50000,epsilon = 0.35856126103453967, C= 4.366473592979633,verbose = 10)

In [23]:
pipeline = pipeline_fit_predict(svr_lin_daily, categoric, numeric, x_train, y_train, x_test, y_test)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


MODEL SCORES:
MAE:  24.244
MSE:  4447.861
RMSE:  66.692
Accuracy: 38.5 %
R2:  56.870 %
Max Residual Error:  1436.930


<hr style="border: 0.5px solid #8888;">


## Hourly Demand - hexagon low resolution 

In [ ]:
train_val_test

In [ ]:
svr_lin_daily = SVR(kernel = 'linear',cache_size=15000, max_iter=50000,epsilon =, C= )

In [ ]:
pipeline = pipeline_fit_predict(svr_lin_daily, categoric, numeric, x_train_val, y_train_val, x_test, y_test)

<hr style="border: 0.5px solid #8888;">


## Hourly Demand - hexagon medium resolution

In [ ]:
train_val_test

In [ ]:
svr_lin_daily = SVR(kernel = 'linear',cache_size=15000, max_iter=50000,epsilon =, C= )

In [ ]:
pipeline = pipeline_fit_predict(svr_lin_daily, categoric, numeric, x_train, y_train, x_test, y_test)

<hr style="border: 0.5px solid #8888;">


## Hourly Demand - hexagon high resolution

In [ ]:
train_val_test

In [ ]:
svr_lin_daily = SVR(kernel = 'linear',cache_size=15000, max_iter=50000,epsilon =, C= )

In [ ]:
pipeline = pipeline_fit_predict(svr_lin_daily, categoric, numeric, x_train, y_train, x_test, y_test)